# Holder Agent

In [ ]:
import requests
import json
import qrcode
import uuid
import base64
import re
import jwt
import time
# https://pypi.org/project/termcolor/
from termcolor import colored,cprint

In [ ]:
# Set the URL for the API endpoint
# Need to specify the host as host.docker.internal as the Prism Agents and the Prism Playground are running in two isolated docker environments
base_url = "http://host.docker.internal:8090"

headers = {'Content-Type': 'application/json'}

## Check Agent Records
1. Connections
2. Credentials
3. Proof Presentations

In [ ]:
print(colored('Agent State:\n', attrs=['bold']))
path = '/prism-agent/connections'

# Make the request
response = requests.get(base_url + path, headers=headers)

print(colored('Connection Records:\n', attrs=['bold']))
# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    
path = '/prism-agent/issue-credentials/records'

print(colored('Credential Records:\n', attrs=['bold']))
# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)
    
path = '/prism-agent/present-proof/presentations'

print(colored('Proof Presentation Records:\n', attrs=['bold']))
# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

## Receive a Connection from Issuer Agent

## Fetch invitationUrl from Issuer Agent

In [ ]:
%store -r invitationUrlbase64
print(colored('Base64 Encoded Raw Connection Invitation Payload:\n', attrs=['bold']))
print(invitationUrlbase64)

# Standard Base64 Decoding
decodedBytes = base64.b64decode(invitationUrlbase64)
decodedStr = str(decodedBytes, "utf-8")

print(colored('\nBase64 Decoded Raw Connection Invitation Payload:\n', attrs=['bold']))
print(decodedStr)


## Accept an OOB connection invitation from Issuer Agent
> Replace `{RAW_INVITATION}` with the value of the '_oob' query string parameter from the invitation URL above
```bash
curl -X 'POST' \
	'http://localhost:8090/prism-agent/connection-invitations' \
	-H 'Content-Type: application/json' \
	-d '{
		"invitation": "{RAW_INVITATION}"
	}' | jq
```

In [ ]:
path = '/prism-agent/connection-invitations'
data = {"invitation": invitationUrlbase64}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    invitation = json.loads(response.content.decode())
    invitationUrl = invitation['invitation']['invitationUrl']
    # invitationUrlbase64 = invitation['invitation']['invitationUrl'].split('=')[1]
    connection_id = invitation['connectionId']
    STATE = loaded_json['state']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

## Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8090/prism-agent/connections' | jq
```

In [ ]:
# STATE = None

path = '/prism-agent//connections/' + connection_id

print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseReceived':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("State: {0}".format(STATE), "blue", attrs=["bold"]))

        try:
            their_did = loaded_json['theirDid']
        except (KeyError) as e:
            print(colored("Holder didn't accept invitation yet".format(connection_id,STATE), "red", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))



## Receive Credentials from Issuer

### Retrieving the list of issue records
```bash
curl -X 'GET' 'http://localhost:8090/prism-agent/issue-credentials/records' | jq
```

In [ ]:
path = '/prism-agent/issue-credentials/records'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    record_id = loaded_json['items'][0]['recordId']
    STATE = loaded_json['items'][0]['protocolState']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Accepting the credential offer from Issuer

Replace `{RECORD_ID}` with the UUID of the record from the previous list
```bash
curl -X 'POST' 'http://localhost:8090/prism-agent/issue-credentials/records/{RECORD_ID}/accept-offer' | jq
```

In [ ]:
%store -r credential_record_id
print("Issuer credential recordId:", credential_record_id)
print("Holder credential recordId:",record_id)

In [ ]:
print(colored("Current state for Credential Issuance {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))

if STATE == 'OfferReceived':
    path = '/prism-agent/issue-credentials/records/' + record_id + '/accept-offer'

    # Make the request
    response = requests.post(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['protocolState']
    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        print(response.content)
    
    
path = '/prism-agent/issue-credentials/records'

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for Credential Issuance {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'CredentialReceived':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['items'][0]['protocolState']
        print(colored("ConnectionId {0}".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("Credential Issuance State: {0}".format(STATE), "blue", attrs=["bold"]))

        # try:
        #     their_did = loaded_json['theirDid']
        # except (KeyError) as e:
        #     print(colored("Holder didn't accept invitation yet".format(connection_id,STATE), "red", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("CredentialId: {0} issued.".format(connection_id), "green", attrs=["bold"]))


### Decode JWT in Credential

In [ ]:
token = loaded_json['items'][0]['jwtCredential']
# print(token)

padded = token + "="*divmod(len(token),4)[1]
# print(padded)
jsondata = base64.urlsafe_b64decode(token)
# print(jsondata)
b64decoded_token = json.loads(jsondata.decode())
print(b64decoded_token,'\n')
header = jwt.get_unverified_header(b64decoded_token)
print (header,'\n')
# print (header['typ'])
# print (header['alg'])


# decodedJWT = jwt.decode(token, algorithms=[header['alg'], ])
# try:
#     jwt_decoded_id_token = jwt.decode(
#         token,
#         jwt_public_key,
#         # audience = config_idporten["aud"],
#         algorithms=[header['alg'], ])
#     print(jwt_decoded_id_token)
# except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
#     print("[ERROR]", e)

try:
    jwt_decoded_id_token = jwt.decode(b64decoded_token, options={"verify_signature": False})
    # print(jwt_decoded_id_token)
    print(json.dumps(jwt_decoded_id_token, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)
    

## Head over to Verifier Notebook to crate connection invitation

## Create connection with verifier

## Fetch invitationUrl from Verifier Agent

In [ ]:
%store -r invitationUrlbase64verifier
print(invitationUrlbase64verifier)

## Accept an OOB connection invitation from Verifier Agent
> Replace `{RAW_INVITATION}` with the value of the '_oob' query string parameter from the invitation URL above
```bash
curl -X 'POST' \
	'http://localhost:8090/prism-agent/connection-invitations' \
	-H 'Content-Type: application/json' \
	-d '{
		"invitation": "{RAW_INVITATION}"
	}' | jq
```

In [ ]:
path = '/prism-agent/connection-invitations'
data = {"invitation": invitationUrlbase64verifier}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    invitation = json.loads(response.content.decode())
    invitationUrl = invitation['invitation']['invitationUrl']
    # invitationUrlbase64 = invitation['invitation']['invitationUrl'].split('=')[1]
    connection_id = invitation['connectionId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

## Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8090/prism-agent/connections' | jq
```

In [ ]:
STATE = None

path = '/prism-agent//connections/' + connection_id

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseReceived':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("State: {0}".format(STATE), "blue", attrs=["bold"]))

        try:
            their_did = loaded_json['theirDid']
        except (KeyError) as e:
            print(colored("Holder didn't accept invitation yet".format(connection_id,STATE), "red", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))



## Verify Credentials

### Retrieving the list of presentation records

```shell
curl -X 'GET' 'http://localhost:8090/prism-agent/present-proof/presentations' -H 'accept: application/json' | jq
```

In [ ]:
path = '/prism-agent/present-proof/presentations'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    presentationId = loaded_json[0]['presentationId']
    STATE = loaded_json[0]['status']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

### Retrieving the list of credentials records
> choose the `{RECORD_ID}` for credential with status CredentialRecieved

```shell
curl -X 'GET' 'http://localhost:8090/prism-agent/issue-credentials/records' -H 'accept: application/json' | jq
```

In [ ]:
print(record_id)
print(presentationId)

### Accepting the Presentation Request 
>Replace `{PRESENTATION_ID}` with the UUID of the record from the presentation records list

>Replace `{RECORD_ID}` with the UUID of the record from the credential records list


```shell
curl -X 'PATCH' \
  'http://localhost:8090/prism-agent/present-proof/presentations/{PRESENTATION_ID}' \
  -H 'Content-Type: application/json' \
  -d '{
  "action": "request-accept",
  "proofId": ["{RECORD_ID}"]
}'
```

In [ ]:
data = {
  "action": "request-accept",
  "proofId": [record_id]
}

path = '/prism-agent/present-proof/presentations/' + presentationId

# Make the request
response = requests.patch(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    print(response.content)
    # loaded_json = json.loads(response.content.decode())
    # print(json.dumps(loaded_json, indent=2))
    # proof_requestId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

In [ ]:
path = '/prism-agent/present-proof/presentations'

print('Current state for Presentation {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'PresentationSent':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json[0]['status']
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("State: {0}".format(STATE), "blue", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("Proof Presentation send to Verifier".format(connection_id), "green", attrs=["bold"]))



# TODO
> Fix response for successful API in PATCH HTTP call above